In [ ]:
import duckdb
con = duckdb.connect("c:/Code/Local Code/deadlock_match_prediction/data/deadlock.db")
raw_con = duckdb.connect("c:/Code/Local Code/deadlock_match_prediction/match_player_raw.duckdb")

In [ ]:
result = raw_con.execute("SELECT count(*) FROM staging_cleaned").fetchall()
print(result)

In [ ]:
con.close()
raw_con.close()

In [ ]:
match_df = con.execute("SELECT * FROM matches").fetchdf()
match_ids = match_df['match_id'].unique().tolist()

# Register match_df as a temporary DuckDB view
raw_con.register("match_df", match_df)

query = """
    SELECT 
        player_count,
        COUNT(*) AS match_count
    FROM (
        SELECT 
            s.match_id,
            COUNT(*) AS player_count
        FROM staging_cleaned s
        JOIN match_df m ON s.match_id = m.match_id
        GROUP BY s.match_id
    )
    GROUP BY player_count
    ORDER BY player_count
"""

query2 = """
    SELECT COUNT(DISTINCT match_id) from match_df;
"""
result = raw_con.execute(query).fetchdf()
print(result)
result2 = raw_con.execute(query2).fetchdf()
print(result2)

In [ ]:
    """Formats raw match_info to be combined with raw_match_players
    Combined data will live in deadlock.db.player_matches_history table
    """
raw_con.execute(f"""
        CREATE OR REPLACE TABLE high_quality_matches AS
        SELECT match_id, start_time, game_mode, match_mode, won
        FROM 'match_info_history'
        WHERE (average_badge_team0 + average_badge_team1) / 2 > 75
    """)
    result = raw_con.execute("Select count(*) from high_quality_matches").fetchone()[0]
    print(f"High quality matches count: {result}")

In [ ]:
raw_con.execute("CREATE OR REPLACE TABLE matches_for_training AS SELECT * FROM match_df")


In [ ]:
result = raw_con.execute("select * from staging_cleaned limit 1").fetchdf()
print(result)

In [ ]:
result = con.execute("SELECT count(distinct account_id) FROM player_matches").fetchdf()
print(result)

In [ ]:
result = raw_con.execute("""SELECT COUNT(*)
FROM staging_cleaned
WHERE match_id IN (
    SELECT DISTINCT match_id FROM matches_for_training
);""").fetchall()
print(result)

In [ ]:
raw_con.execute("""SELECT COUNT(*) AS non_unique_match_ids
FROM (
    SELECT s.account_id
    FROM staging_cleaned s
    JOIN matches_for_training m ON s.match_id = m.match_id
    GROUP BY s.account_id
    HAVING COUNT(*) > 1
);""").fetchall()

*** Creates / replaces filtered_player_matches, completing with data from staging_cleared, where match_id matches matches_for_Training

This should create match history for all players in matches_for_training

In [ ]:
raw_con.execute("""
    CREATE OR REPLACE TABLE filtered_player_matches AS
SELECT
    s.account_id,
    s.match_id,
    s.hero_id,
    s.team,
    s.kills,
    s.deaths,
    s.assists,
    s.denies,
    s.net_worth,
    s.won
FROM staging_cleaned s
WHERE s.account_id IN (
    SELECT DISTINCT account_id 
    FROM dup_player_matches
);
                """)

In [ ]:
query1 = raw_con.execute("SELECT count(distinct match_id) from filtered_player_matches").fetchall()
query2 = raw_con.execute("SELECT count(*) from filtered_player_matches").fetchall()
print(query1)
print(query2)

In [ ]:
raw_con.execute("""
    SELECT 
    player_count,
    COUNT(*) AS match_count
    FROM (
    SELECT 
        match_id,
        COUNT(account_id) AS player_count
    FROM filtered_player_matches
    GROUP BY match_id
)
GROUP BY player_count
ORDER BY player_count;                
                """).fetchall()

In [ ]:
import duckdb
import matplotlib.pyplot as plt

df = raw_con.execute("""
    SELECT 
        account_id,
        COUNT(match_id) AS total_matches
    FROM filtered_player_matches
    GROUP BY account_id
""").fetchdf()

# Histogram for total matches
plt.figure(figsize=(10, 6))
plt.hist(df['total_matches'], bins=50, edgecolor='black')
plt.title('Distribution of Total Matches per Account')
plt.xlabel('Total Matches')
plt.ylabel('Number of Players')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
# Fetch total match counts per player
df = raw_con.execute("""
    SELECT account_id, COUNT(*) AS total_matches
    FROM filtered_player_matches
    GROUP BY account_id
""").fetchdf()

# Calculate 95th percentile
p95 = np.percentile(df["total_matches"], 95)

# Plot
plt.figure(figsize=(10, 5))
plt.hist(df[df["total_matches"] <= p95]["total_matches"], bins=50)
plt.title("Player Total Matches (≤95th percentile)")
plt.xlabel("Total Matches")
plt.ylabel("Number of Players")
plt.grid(True)
plt.show()

In [ ]:
df = raw_con.execute("""SELECT COUNT(*) 
FROM staging_cleaned s
INNER JOIN matches_for_training m ON s.match_id = m.match_id;""").fetchdf()
print(f"{df.columns}\n\n {len(df)}")

In [ ]:
    raw_con.execute("""
        CREATE TABLE dup_player_matches (
        account_id BIGINT,
        match_id BIGINT,
        hero_id INTEGER,
        team VARCHAR,
        kills INTEGER,
        deaths INTEGER,
        assists INTEGER,
        denies INTEGER,
        net_worth BIGINT,
        won BOOLEAN,
        PRIMARY KEY (account_id, match_id)
        )
        """)

In [ ]:
raw_con.execute("""drop table dup_player_matches""")

Duplicate main.player_matches into raw_data.dup_player_matches

In [ ]:
df = con.execute("""
    select * from player_matches""").fetchdf()
print(f"{df.columns}\n\n {len(df)}")
raw_con.execute("""
    INSERT into dup_player_matches
    SELECT * FROM df
                """)

In [ ]:
result = raw_con.execute("SELECT count(*) from dup_player_matches").fetchdf()
print(result)

In [ ]:
con.execute("""
    UPDATE player_matches
    SET won = (
        SELECT CASE 
            WHEN player_matches.team = m.winning_team THEN 1 
            ELSE 0 
            END
        FROM matches m
        WHERE player_matches.match_id = m.match_id
);  """)

In [ ]:
oldest_fpm_match_start = raw_con.execute("""
    SELECT hqm.start_time
    FROM match_info_history hqm
    WHERE hqm.match_id = (
        SELECT MIN(match_id)
        FROM filtered_player_matches
    )
""").fetchone()

oldest_high_quality = raw_con.execute("""
    SELECT MIN(start_time) FROM high_quality_matches
""").fetchone()

print("Start time from lowest match_id:", oldest_fpm_match_start[0])
print("Oldest start time overall:", oldest_high_quality[0])

In [ ]:
result = raw_con.execute("SELECT count(*) from dup_player_matches limit 1").fetchdf()
print(result)

Count and display layout of multiple tables

In [ ]:
result = raw_con.execute("SELECT COUNT(*) FROM matches_for_training").fetchdf()
print(f"matches_for_training: {result}")
result = raw_con.execute("SELECT * FROM matches_for_training LIMIT 1").fetchdf()
print(f"matches_for_training: {result}")

result = raw_con.execute("SELECT COUNT(*) FROM dup_player_matches").fetchdf()
print(f"\n\ndup_player_matches: {result}")
result = raw_con.execute("SELECT * FROM dup_player_matches LIMIT 1").fetchdf()
print(f"dup_player_matches: {result}")

result = raw_con.execute("SELECT COUNT(*) FROM filtered_player_matches").fetchdf()
print(f"\n\nfiltered_player_matches: {result}")
result = raw_con.execute("SELECT * FROM filtered_player_matches LIMIT 1").fetchdf()
print(f"filtered_player_matches: {result}")

result = raw_con.execute("SELECT COUNT(*) FROM high_quality_matches").fetchdf()
print(f"\n\nhigh_quality_matches: {result}")
result = raw_con.execute("SELECT * FROM high_quality_matches LIMIT 1").fetchdf()
print(f"high_quality_matches: {result}")

result = raw_con.execute("SELECT COUNT(*) FROM staging_cleaned").fetchdf()
print(f"\nstaging_cleaned: {result}")
result = raw_con.execute("SELECT * FROM staging_cleaned LIMIT 1").fetchdf()
print(f"staging_cleaned: {result}")

## How many matches of history for each player in filtered_player_matches
count occurance of each account_id in filtered_player_matches


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Fetch data
result = raw_con.execute("""
    SELECT account_id, COUNT(*) AS match_count
    FROM filtered_player_matches
    GROUP BY account_id
""").fetchdf()

# Bin match counts into buckets of 50
bins = range(0, result['match_count'].max() + 9, 10)
result['match_bucket'] = pd.cut(result['match_count'], bins=bins, right=False)

# Count players in each bucket
bucket_counts = result['match_bucket'].value_counts().sort_index()

# Plot
plt.figure(figsize=(12, 6))
plt.bar(bucket_counts.index.astype(str), bucket_counts.values)
plt.xticks(rotation=45, ha='right')
plt.xlabel('Match Count Bucket')
plt.ylabel('Number of Players')
plt.title('Player Distribution by Match Count Buckets (50 per bucket)')
plt.tight_layout()
plt.show()

# Combine data and insert into main.player_matches_history
for match_id in filtered_player_matches
Join match_info.match_id with filtered_player_matches.match_id
account_id, match_id, hero_id, team, kills, deaths, assists,
denies, net_worth, won
